In [1]:
import itertools
from treys import Card
from treys import Evaluator
from treys import Deck

In [2]:
def lget(l, index, default=None):
    try:
        return l[index]
    except IndexError:
        return default

In [3]:
def typeOfCard(card, cardsDesired):
    if not cardsDesired:
        return True
    
    return card in cardsDesired

In [90]:
%reload_ext autoreload
%autoreload 2
from strategies import firstNOfAKind

# match keywords with parameters for function (N, additionalPair)
validKeyWords = {'firstFive': (1,0), 'firstPair': (2,0), 'firstTwoPair': (2,1),
         'firstTrips': (3,0), 'firstFullHouse': (3, 1), 'firstQuads': (4,0)}

def simulatePokerSolitaire(cardsDesired, keyWord = None):
    playerCards, communityCards, deck = [], [], Deck()

    if keyWord is None:
        # run with cardsDesired
        while len(playerCards) < 5 or len(communityCards) < 5:
            card = deck.draw(1)
            if len(playerCards) == 5:
                communityCards.append(card)
            elif typeOfCard(Card.print_pretty_card(card), Card.print_pretty_cards(cardsDesired)):
                playerCards.append(card)
            else:
                communityCards.append(card)
    elif keyWord not in validKeyWords:
        raise SyntaxError("Invalid keyword")
    else:
        N, additionalPair = validKeyWords[keyWord]
        keyWord = 'firstNOfAKind' + '(deck, {}, {})'.format(N, additionalPair)
        playerCards, communityCards = eval(keyWord)

    return playerCards, communityCards

In [21]:
def findBestCombination(totalCombinations, highestScore, evaluator):
    for combination in totalCombinations:
        if evaluator.evaluate(list(combination), []) == highestScore:
            return list(combination)
    
    return None

In [22]:
def evaluateBetterHand(playerCards, communityCards):
    evaluator = Evaluator()
    playerScore = evaluator.evaluate(playerCards, [])
    totalCombinations = [combinationCommunityCards for combinationCommunityCards in itertools.combinations(communityCards, 5)]
    highestScore = min(evaluator.evaluate(list(combination), []) for combination in totalCombinations)
    bestCommunityCombination = findBestCombination(totalCombinations, highestScore, evaluator)
    
    print(playerScore, highestScore)
    return highestScore >= playerScore, bestCommunityCombination

In [98]:
# note CHAR_RANK_TO_INT_RANK function doesn't exist

def playPoker(enterHand, keyWord = None):
    numericCardHand = [((CHAR_RANK_TO_INT_RANK[lget(card, 0)] << 8) + 
                        (CHAR_SUIT_TO_INT_SUIT[lget(card, 1)] << 12)) for card in enterHand]
    playerCards, communityCards = simulatePokerSolitaire(numericCardHand, keyWord)
    playerWin, bestCommunityCombination = evaluateBetterHand(playerCards, communityCards)
    
    return playerWin, Card.print_pretty_cards(playerCards), Card.print_pretty_cards(communityCards), Card.print_pretty_cards(bestCommunityCombination)

In [43]:
enterHand = ['4d', '3s', 'As', '2d', '5c']


1

In [100]:
playPoker([], "firstFullHouse")




177 2712


(True,
 ' [3♠],[A♦],[A♥],[3♣],[A♣] ',
 ' [9♠],[T♥],[J♥],[K♥],[4♥],[2♣],[2♠],[K♣] ',
 ' [J♥],[K♥],[2♣],[2♠],[K♣] ')